In [2]:
from elasticsearch import Elasticsearch
import re
es = Elasticsearch()

In [3]:
syslog_count = es.count(index="homequitybank-2016.09.01", doc_type="syslog")['count']
syslog_count

441016

In [4]:
res = es.search(index="homequitybank-2016.09.01", doc_type="syslog",size=1, from_=10, body={"query": {"match_all":{}}})
res

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVbjVhxqh76HlwhVdPg6',
    '_index': 'homequitybank-2016.09.01',
    '_score': 1.0,
    '_source': {'@timestamp': '2016-09-01T01:21:25.807Z',
     '@version': '1',
     'customer': 'homequitybank',
     'host': '172.16.4.250',
     'message': '<85>1 2016-08-31T21:21:07--4:00 192.168.0.11 CP-GW - Log [Fields@1.3.6.1.4.1.2620 Action="accept" UUid="{0x57c78283,0x1,0x20410ac,0xc0000000}" inzone="External" outzone="Internal" rule="22" rule_uid="{7CF5AFC9-7850-47EF-9E2F-42A332564278}" rule_name="Internet to Kace (Saturn)" service_id="KACE_High_Port" src="205.200.185.167" dst="68.71.192.203" proto="6" xlatedst="10.0.4.10" NAT_rulenum="31" NAT_addtnl_rulenum="1" dst_user_name="kbox_user (kbox_user)(+)Miro Zgavc (mzgavc)(+)" product="VPN-1 & FireWall-1" service="52230" s_port="54166" product_family="Network"]',
     'type': 'syslog'},
    '_type': 'syslog'}],
  'max_score': 1.0,
  'total': 441016},
 'timed_out':

In [5]:
res1 = es.search(index="homequitybank-2016.09.01", doc_type="syslog",size=1, from_=10, body={
  "query": { "bool" : { "must" : { "match": { "message":"product_family=\"Network\"" } } } } })
print(res1)

{'_shards': {'successful': 5, 'failed': 0, 'total': 5}, 'hits': {'max_score': 0.27120787, 'hits': [{'_type': 'syslog', '_index': 'homequitybank-2016.09.01', '_source': {'@timestamp': '2016-09-01T01:45:46.130Z', 'type': 'syslog', '@version': '1', 'host': '172.16.1.3', 'message': '<85>1 2016-08-31T21:45:44--4:00 192.168.1.11 CP-GW - Log [Fields@1.3.6.1.4.1.2620 Action=" " UUid="{0x57c78848,0x0,0xb01a8c0,0x749507b6}" roles="Anyuser" product="VPN-1 & FireWall-1" product_family="Network"]', 'customer': 'homequitybank'}, '_id': 'AVbjbGuGh76HlwhVdX6v', '_score': 0.27120787}], 'total': 441006}, 'timed_out': False, 'took': 78}


In [18]:
all_keys = set([])
for x in range(10000):
#     res = es.search(index="homequitybank-2016.09.01", doc_type="syslog",size=1, from_=x, body={
#   "query": { "match": { "message":"src=" , "message":"dst"} } } )
    res = es.search(index="homequitybank-2016.09.01", doc_type="syslog",size=1, from_=x, body={"query": {"match_all":{}}})
    syslog = res['hits']['hits'][0]['_source']['message']
    kv = re.findall("(\w*=\".*?\")",syslog)
    sys_dict = {l[0]: l[1].strip("\"") for l in [l.split("=") for l in kv]}
    key_set = set([])
    for key in sys_dict:
        key_set.add(key)
    all_keys.add(frozenset(key_set))

In [21]:
all_keys

{frozenset({'Action',
            'NAT_addtnl_rulenum',
            'NAT_rulenum',
            'UUid',
            'dst',
            'inzone',
            'outzone',
            'product',
            'product_family',
            'proto',
            'rule',
            'rule_uid',
            's_port',
            'service',
            'service_id',
            'snid',
            'src',
            'src_machine_name',
            'xlatesport',
            'xlatesrc'}),
 frozenset({'Action',
            'UUid',
            'dst',
            'dst_user_name',
            'inzone',
            'outzone',
            'product',
            'product_family',
            'proto',
            'rule',
            'rule_name',
            'rule_uid',
            's_port',
            'service',
            'src'}),
 frozenset({'Action',
            'NAT_addtnl_rulenum',
            'NAT_rulenum',
            'UUid',
            'dst',
            'dst_user_name',
            'inzone',
    

In [11]:
res1 = es.search(index="homequitybank-2016.09.01", doc_type="syslog",size=1, from_=10, body={
  "query": { "bool" : { "must" : { "match": { "message":"app_risk" } } } } })
res1

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVbjWnNrh76HlwhVdRHp',
    '_index': 'homequitybank-2016.09.01',
    '_score': 0.6642222,
    '_source': {'@timestamp': '2016-09-01T01:26:10.207Z',
     '@version': '1',
     'customer': 'homequitybank',
     'host': '172.16.4.250',
     'message': '<85>1 2016-08-31T21:25:52--4:00 192.168.0.11 CP-GW - Log [Fields@1.3.6.1.4.1.2620 Action="allow" UUid="{0x57c77337,0x1,0x20410ac,0xc0000000}" src="172.16.3.111" dst="23.198.239.186" proto="6" appi_name="******" app_desc="******" app_id="******" app_category="******" matched_category="******" app_properties="******" app_risk="******" app_rule_id="******" app_rule_name="******" app_sig_id="60095476:15" proxy_src_ip="172.16.3.111" user="Sertan Sahbaz (ssahbaz)(+)" src_user_name="Sertan Sahbaz (ssahbaz)(+)" src_machine_name="cfnz902@chip.ca" snid="64c1b7b0" product="Application Control" service="443" s_port="63447" product_family="Network"]',
     'type': 'syslo